# Step 0: 環境構築とライブラリのインポート

このノートブック全体の処理を実行するために必要な,すべてのPythonライブラリをインストールし,インポートします.

### \#\# 実行内容 ⚙️

1.  **ライブラリの一括インストール**:

      * `!pip install -r requirements.txt` コマンドが,`requirements.txt`に記載されたすべてのライブラリを一度にインストールします.

2.  **ライブラリのインポート**:

      * インストールしたライブラリや,Pythonに標準で組み込まれているライブラリ（`os`, `json`など）を,このノートブックのメモリに読み込み,後続のセルで使えるようにします.

In [ ]:
!pip install -r requirements.txt

In [ ]:
import json
import os
import re
import shutil
import subprocess
import sys

import requests

# Step 1: リポジトリのクローン

このノートブックの最初のステップとして. SBOMの分析対象となる複数のGitリポジトリを自動的にクローンします.

### \#\# 事前準備 📝

**`url_list.txt`** という名前のファイルをこのノートブックと同じ階層に作成してください.
ファイルの中には. クローンしたいリポジトリのURLを1行に1つずつ記述します.

**`url_list.txt` の記述例:**

```
https://github.com/user1/repo1.git
https://github.com/user2/repo2.git
https://github.com/user3/another-repo.git
```

### \#\# 実行内容 ⚙️

以下のコードは. `url_list.txt` を読み込み. 各URLに対して `git clone` コマンドを実行します.

  * クローンされたリポジトリは. **`cloned_repositories`** ディレクトリ内に保存されます.
  * 既に同名のリポジトリが存在する場合は. 時間短縮のためクローン処理をスキップします.
  * クローンの進捗状況はリアルタイムで表示されます.

In [ ]:
# --- 設定項目 ---
# クローン対象のリポジトリURLリストファイル
url_file_path = 'url_list.txt'
# リポジトリのクローン先ディレクトリ
clone_to_directory = 'cloned_repositories'

# --- 処理の開始 ---
print(f"--- Starting: Cloning repositories into '{clone_to_directory}' ---")

# 保存先ディレクトリを作成する
os.makedirs(clone_to_directory, exist_ok=True)
print("-" * 50)

# URLリストファイルを読み込む
try:
    with open(url_file_path, 'r') as file:
        # 空行を除外してリスト化する
        urls = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print(f"❌ Error: '{url_file_path}' was not found.")
    print("Please make sure the file exists in the same directory as the notebook.")
    urls = [] # エラー時はリストを空にする

for repo_url in urls:
    # --- 既存リポジトリのスキップ処理 ---
    # URLからリポジトリ名を取得する
    repo_name = repo_url.split('/')[-1].replace('.git', '')
    # リポジトリの保存パスを構築する
    repo_path = os.path.join(clone_to_directory, repo_name)
    
    # ディレクトリが既に存在する場合,処理をスキップする
    if os.path.isdir(repo_path):
        print(f"🟢 Skipping: {repo_name} (Directory already exists)")
        print("-" * 50)
        continue

    print(f"Cloning: {repo_url}")
    try:
        # git cloneプロセスを開始し,進捗をリアルタイムで表示する
        # --progressフラグで進捗表示を強制し,stderrから出力を受け取る
        process = subprocess.Popen(
            ['git', 'clone', '--progress', repo_url],
            cwd=clone_to_directory,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            encoding='utf-8',
            errors='replace' # エンコーディングエラーを回避
        )

        # 標準エラーを1行ずつ読み込み,進捗として表示する
        while process.poll() is None:
            line = process.stderr.readline()
            if line:
                # カーソルを行頭に戻し,進捗表示を上書きする
                print(f"   {line.strip()}", end='\r')
        
        # 最後の進捗表示行をクリアする
        print(" " * 80, end="\r")

        # クローンの成否を判定する
        if process.returncode == 0:
            print("✅ Clone successful.")
        else:
            # エラー内容を取得して表示する
            stdout_err, stderr_err = process.communicate()
            error_message = stderr_err if stderr_err else stdout_err
            print(f"❌ Error cloning. Reason: {error_message.strip()}")

    except FileNotFoundError:
        print("❌ Error: 'git' command not found. Please install Git and ensure it is in your system's PATH.")
        break # Gitがない場合は処理を中断
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("-" * 50)

print("Clone process finished.")

# Step 2: SBOMの生成 (sbom-tool)

前のステップでクローンした各リポジトリに対して,Microsoftの **`sbom-tool`** を使用してベースとなるSBOMを生成します.このツールは,リポジトリ内の全ファイルをスキャンし,ハッシュ値を含む詳細なファイル情報を取得することに優れています.

### ## 事前準備 📝

* **`sbom-tool`** がシステムにインストールされ,PATHが通っている必要があります.
* Step 1が完了しており,**`cloned_repositories`** ディレクトリ内に分析対象のリポジトリが存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`cloned_repositories`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  **パラメータの自動取得**:
    * `git`コマンドを実行し,現在のコミットハッシュを **`packageVersion`** として取得します.
    * `.git/config`ファイルを解析し,リポジトリの所有者（OrganizationまたはUser）を **`packageSupplier`** として自動で設定します.

2.  **`sbom-tool` の実行**:
    * 取得したパラメータ（パッケージ名,バージョン,サプライヤー）を使って`sbom-tool generate`コマンドを動的に構築し,実行します.
    * これにより,各リポジトリのルートに **`_manifest`** ディレクトリが生成されます.

3.  **成果物の移動**:
    * 生成された`_manifest`ディレクトリを,後続の処理で扱いやすいように **`generated_sboms/<リポジトリ名>/source/`** ディレクトリ内に移動します.

すでに成果物が存在する場合,この処理はスキップされます.

In [ ]:
# --- 設定項目 ---
clone_to_directory = 'cloned_repositories'
sbom_output_directory = 'generated_sboms'

# --- 処理の開始 ---
print(f"--- Starting: Generating and moving SBOMs ---")
print("-" * 50)

original_path = os.getcwd()
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    # クローンディレクトリ内のリポジトリ一覧を取得する
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
        print("No repositories found to run commands on.")

    for repo_name in repo_dirs:
        # 最終的な保存先パスを定義する
        final_repo_dir = os.path.join(original_path, sbom_output_directory, repo_name)
        final_manifest_path = os.path.join(final_repo_dir, 'source', '_manifest')

        # 既に成果物が存在する場合は処理をスキップする
        if os.path.isdir(final_manifest_path):
            print(f"🟢 Skipping: {repo_name} (SBOM manifest already exists)")
            print("-" * 50)
            continue

        repo_path = os.path.join(clone_to_directory, repo_name)
        print(f"▶️  Entering: {repo_name}")
        
        try:
            os.chdir(repo_path)
            
            # --- パラメータ取得 ---
            package_name = repo_name
            git_version_result = subprocess.run(['git', 'rev-parse', 'HEAD'], capture_output=True, text=True, check=True)
            package_version = git_version_result.stdout.strip()
            
            # .git/configからサプライヤー（所有者）情報を抽出する
            package_supplier = "Unknown"
            with open('.git/config', 'r') as config_file:
                match = re.search(r'url\s*=\s*https?://github\.com/([^/]+)/', config_file.read())
                if match:
                    package_supplier = match.group(1)
            
            print(f"   Package Name: {package_name}")
            print(f"   Package Version: {package_version[:12]}...")
            print(f"   Package Supplier: {package_supplier}")
            
            # 既存のSBOM出力ディレクトリがあれば削除する
            if os.path.isdir('_manifest'):
                print("   Found existing '_manifest' directory. Removing it.")
                shutil.rmtree('_manifest')

            # --- sbom-tool実行 ---
            command = [
                'sbom-tool', 'generate', '-bc', '.', '-b', '.',
                '-pn', package_name, '-pv', package_version, '-ps', package_supplier
            ]
            print(f"   Executing: {' '.join(command)}")
            subprocess.run(command, check=True, capture_output=True, text=True)
            print("✅ SBOM generation successful.")

            # --- 移動処理 ---
            source_manifest_path = '_manifest'
            
            if os.path.isdir(source_manifest_path):
                # 'source' ディレクトリを作成し,その中に成果物を移動する
                destination_dir = os.path.join(final_repo_dir, 'source')
                os.makedirs(destination_dir, exist_ok=True)
                
                print(f"   Moving '{source_manifest_path}' into: {destination_dir}")
                shutil.move(source_manifest_path, destination_dir)
                print("✅ Move successful.")
            else:
                print(f"⚠️ Warning: Could not find generated manifest directory at '{source_manifest_path}'")

        except FileNotFoundError:
            print(f"❌ Error: The command 'sbom-tool' or 'git' was not found.")
            break
        except subprocess.CalledProcessError as e:
            print(f"❌ Error executing command in {repo_name}.")
            if e.stdout:
                print(f"   [stdout]:\n{e.stdout.strip()}")
            if e.stderr:
                print(f"   [stderr]:\n{e.stderr.strip()}")
        finally:
            os.chdir(original_path)
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")

print("All processes finished.")

# Step 3: SBOMの生成 (Syft)

`Step 1`でクローンした各リポジトリに対し,**`syft`** を使用してSBOMを生成します.`syft`は,`requirements.txt`やGitHub Actionsのワークフローファイルなど,様々なパッケージマニフェストから依存関係を検出することに優れています.

### ## 事前準備 📝

* **`syft`** がシステムにインストールされ,PATHが通っている必要があります.
* Step 1が完了しており,**`cloned_repositories`** ディレクトリ内に分析対象のリポジトリが存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`cloned_repositories`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  **`syft` の実行**:
    * 各リポジトリのディレクトリに移動し,`syft dir:./ -o spdx-json`コマンドを実行します.
    * コマンドの標準出力をキャプチャし,リポジトリ内に`syft-sbom.json`という名前の一時ファイルとして保存します.

2.  **成果物の移動**:
    * 生成された`syft-sbom.json`を,後続の処理で扱いやすいように **`generated_sboms/<リポジトリ名>/source/`** ディレクトリ内に移動します.

すでに`syft-sbom.json`が最終的な保存先に存在する場合,そのリポジトリの処理はスキップされます.

In [ ]:
# --- 設定項目 ---
clone_to_directory = 'cloned_repositories'
sbom_output_directory = 'generated_sboms'

# --- 処理の開始 ---
print(f"--- Starting: Generating SBOMs with Syft ---")
print("-" * 50)

# 実行前のカレントディレクトリを保存する
original_path = os.getcwd()
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    # クローンディレクトリ内のリポジトリ一覧を取得する
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
        print("No repositories found to run commands on.")

    for repo_name in repo_dirs:
        repo_path = os.path.join(clone_to_directory, repo_name)
        
        try:
            # --- 最終的なファイルパスを定義 ---
            destination_dir = os.path.join(original_path, sbom_output_directory, repo_name, 'source')
            final_sbom_filename = "syft-sbom.json"
            final_sbom_path = os.path.join(destination_dir, final_sbom_filename)

            # --- 処理をスキップするかの判定 ---
            # 既に成果物が存在する場合は処理をスキップする
            if os.path.exists(final_sbom_path):
                print(f"🟢 Skipping: {repo_name} (SBOM file already exists)")
                print("-" * 50)
                continue

            print(f"▶️  Entering: {repo_name}")
            os.chdir(repo_path)
            
            # --- syftコマンドの実行 ---
            temp_sbom_filename = 'syft-sbom.json'
            # 現在のディレクトリをスキャンし,SPDX JSON形式で出力する
            command = ['syft', 'dir:./', '-o', 'spdx-json']
            print(f"   Executing: {' '.join(command)} > {temp_sbom_filename}")
            
            result = subprocess.run(
                command, check=True, capture_output=True, text=True
            )
            print("✅ Syft execution successful.")

            # syftの標準出力を一時ファイルに書き込む
            with open(temp_sbom_filename, 'w', encoding='utf-8') as f:
                f.write(result.stdout)

            # --- 一時ファイルを最終的な場所に移動 ---
            os.makedirs(destination_dir, exist_ok=True)
            print(f"   Moving SBOM to: {final_sbom_path}")
            shutil.move(temp_sbom_filename, final_sbom_path)
            print("✅ SBOM file saved.")

        except FileNotFoundError:
            print(f"❌ Error: The command 'syft' or 'git' was not found.")
            break
        except subprocess.CalledProcessError as e:
            print(f"❌ Error executing command in {repo_name}.")
            if e.stdout:
                print(f"   [stdout]:\n{e.stdout.strip()}")
            if e.stderr:
                print(f"   [stderr]:\n{e.stderr.strip()}")
        finally:
            # カレントディレクトリを元に戻す
            os.chdir(original_path)
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")

print("All processes finished.")

# Step 4: SBOMの取得 (GitHub Dependency Graph API)

このステップでは,GitHubの **Dependency Graph API** を利用して,各リポジトリのSBOM（Software Bill of Materials）を取得します.このAPIから得られるSBOMは,特に**ライセンス**や**著作権情報**が豊富に含まれているという長所があります.

### ## 事前準備 📝

* **Dependency Graphの有効化**: 分析対象のリポジトリで,**Dependency Graph**機能が有効になっている必要があります.これは通常,パブリックリポジトリではデフォルトで有効ですが,プライベートリポジトリでは手動での有効化が必要です.（`Settings` > `Code security and analysis`）
* **`url_list.txt`**: Step 1で作成した,リポジトリのURLが記載されたファイルが必要です.

### ## 実行内容 ⚙️

以下のコードは,`url_list.txt`を読み込み,各リポジトリに対して以下の処理を自動的に実行します.

1.  **APIリクエスト**:
    * URLからリポジトリの所有者と名前を抽出し,適切なAPIエンドポイント（`api.github.com/repos/{owner}/{repo}/dependency-graph/sbom`）を構築します.
    * 構築したURLに対して`GET`リクエストを送信し,SBOMデータをJSON形式で取得します.

2.  **成果物の保存**:
    * APIから正常に取得できたSBOMデータを,**`generated_sboms/<リポジトリ名>/source/dependency-graph-sbom.json`** という名前のファイルとして保存します.

既に成果物ファイルが存在する場合,APIへのリクエストを節約するため,そのリポジトリの処理はスキップされます.

In [ ]:
# --- 設定項目 ---

# 1. GitHubリポジトリのURLリストが書かれたファイル
url_file_path = 'url_list.txt'

# 2. 生成されたSBOM(JSONファイル)を保存するディレクトリ
sbom_output_directory = 'generated_sboms'


# --- 処理の開始 ---
print("--- Starting: Fetching SBOMs from GitHub API (No Token) ---")
print("-" * 50)

original_path = os.getcwd()
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    with open(url_file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print(f"❌ Error: '{url_file_path}' was not found.")
    urls = []

for repo_url in urls:
    # URLからownerとrepoを抽出
    match = re.search(r"github\.com/([^/]+)/([^/.]+)", repo_url)
    if not match:
        print(f"⚠️ Warning: Could not parse owner/repo from URL: {repo_url}")
        continue
    
    owner, repo_name = match.groups()
    
    # --- 保存先のパスを定義し,スキップ判定 ---
    destination_dir = os.path.join(original_path, sbom_output_directory, repo_name, 'source')
    final_sbom_path = os.path.join(destination_dir, 'dependency-graph-sbom.json')

    if os.path.exists(final_sbom_path):
        print(f"🟢 Skipping: {repo_name} (SBOM file already exists)")
        print("-" * 50)
        continue
        
    print(f"▶️  Fetching SBOM for: {owner}/{repo_name}")

    # --- GitHub APIへのリクエスト (認証ヘッダーなし) ---
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/dependency-graph/sbom"
    headers = {
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }

    try:
        response = requests.get(api_url, headers=headers)
        
        if response.status_code == 200:
            print("✅ API request successful.")
            sbom_data = response.json().get('sbom')
            if not sbom_data:
                print(f"❌ Error: 'sbom' key not found in the API response for {repo_name}.")
                continue

            # --- SBOMをファイルに保存 ---
            os.makedirs(destination_dir, exist_ok=True)
            print(f"   Writing SBOM to: {final_sbom_path}")
            
            with open(final_sbom_path, 'w', encoding='utf-8') as f:
                json.dump(sbom_data, f, ensure_ascii=False, indent=2)
            print("✅ SBOM file saved.")

        elif response.status_code == 404:
            print(f"⚠️ Warning: Could not fetch SBOM for {repo_name}. (Status: 404)")
            print("   The repository may not exist, or the Dependency Graph may not be enabled.")
        
        else:
            print(f"❌ Error: Failed to fetch SBOM for {repo_name}. Status code: {response.status_code}")
            print(f"   Response: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Error: A network error occurred while contacting the GitHub API.")
        print(f"   Details: {e}")
    
    finally:
        print("-" * 50)

print("All processes finished.")

# Step 5: SBOMの生成 (Trivy)

このステップでは,オープンソースの脆弱性スキャナである **`Trivy`** を使用して,各リポジトリからSBOMを生成します.`Trivy`は,ファイルシステム内のパッケージ依存関係を高速かつ広範囲に検出する能力に優れています.

### ## 事前準備 📝

* **`Trivy`** がシステムにインストールされ,PATHが通っている必要があります.
* Step 1が完了しており,**`cloned_repositories`** ディレクトリ内に分析対象のリポジトリが存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`cloned_repositories`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  **`Trivy` の実行**:
    * 各リポジトリのディレクトリに移動し,`trivy fs . --format spdx-json --output spdx-json-by-trivy.json` コマンドを実行します.
    * これにより,リポジトリ内に`spdx-json-by-trivy.json`という名前でSBOMファイルが直接生成されます.

2.  **成果物の移動とリネーム**:
    * 生成された`spdx-json-by-trivy.json`を,後続の処理で統一的に扱えるように **`generated_sboms/<リポジトリ名>/source/trivy-sbom.json`** という名前に変更して移動します.

すでに`trivy-sbom.json`が最終的な保存先に存在する場合,そのリポジトリの処理はスキップされます.

In [ ]:
# --- 設定項目 ---

# クローンされたリポジトリの保存ディレクトリ
clone_to_directory = 'cloned_repositories'
# 生成されたSBOMの保存ルートディレクトリ
sbom_output_directory = 'generated_sboms'

# --- 処理の開始 ---
# Trivyを使ったSBOM生成処理開始メッセージを表示
print(f"--- Starting: Generating SBOMs with Trivy ---")
print("-" * 50)

# 現在の作業ディレクトリを保存
original_path = os.getcwd()
# SBOM出力ディレクトリが存在しない場合は作成
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    # クローンディレクトリ内のリポジトリ一覧を取得
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
        print("No repositories found to run commands on.")

    # 各リポジトリに対してTrivyを使ったSBOM生成処理を実行
    for repo_name in repo_dirs:
        repo_path = os.path.join(clone_to_directory, repo_name)
        
        try:
            # --- 最終的なファイルパスを定義 ---
            # SBOMの最終的な保存先ディレクトリを構築
            destination_dir = os.path.join(original_path, sbom_output_directory, repo_name, 'source')
            
            # 生成されるSBOMのファイル名を 'trivy-sbom.json' に固定
            final_sbom_filename = "trivy-sbom.json"
            
            # 最終的なSBOMファイルのフルパスを構築
            final_sbom_path = os.path.join(destination_dir, final_sbom_filename)

            # --- 処理をスキップするかの判定 ---
            # 既にSBOMファイルが存在する場合はスキップ
            if os.path.exists(final_sbom_path):
                print(f"🟢 Skipping: {repo_name} (SBOM file already exists)")
                print("-" * 50)
                continue

            # --- ここからリポジトリ内での処理 ---
            print(f"▶️  Entering: {repo_name}")
            # カレントディレクトリをリポジトリのパスに変更
            os.chdir(repo_path)
            
            # --- trivyコマンドの実行 ---
            # trivyが出力する一時ファイル名を定義
            temp_sbom_filename = 'spdx-json-by-trivy.json'
            # trivyコマンドを構築
            command = ['trivy', 'fs', '.', '--format', 'spdx-json', '--output', temp_sbom_filename]
            print(f"   Executing: {' '.join(command)}")
            
            # trivyコマンドを実行
            # Trivyはファイルに直接出力するため,出力のキャプチャは不要
            subprocess.run(command, check=True, capture_output=True, text=True)
            print("✅ Trivy execution successful.")

            # --- 一時ファイルを最終的な場所に移動 ---
            # 最終的な保存先ディレクトリが存在しない場合は作成
            os.makedirs(destination_dir, exist_ok=True)
            print(f"   Moving SBOM to: {final_sbom_path}")
            # 一時ファイルを最終的な保存先にリネームしながら移動
            shutil.move(temp_sbom_filename, final_sbom_path)
            print("✅ SBOM file saved.")

        except FileNotFoundError:
            # 'trivy'コマンドが見つからない場合のエラー処理
            print(f"❌ Error: The command 'trivy' was not found.")
            break
        except subprocess.CalledProcessError as e:
            # コマンド実行中にエラーが発生した場合の処理
            print(f"❌ Error executing command in {repo_name}.")
            if e.stdout:
                print(f"   [stdout]:\n{e.stdout.strip()}")
            if e.stderr:
                print(f"   [stderr]:\n{e.stderr.strip()}")
        finally:
            # カレントディレクトリを元のパスに戻す
            os.chdir(original_path)
            print("-" * 50)

except FileNotFoundError:
    # クローンディレクトリが見つからない場合のエラー処理
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")

# 全てのTrivyを使ったSBOM生成処理が完了
print("All processes finished.")

# Step 6: ベースSBOMの作成と整形

このステップでは,`Step 2`で`sbom-tool`が出力した`manifest.spdx.json`をコピーし,後続の処理の基礎となる **`combined_sbom.json`** を作成します.同時に,JSONファイルのトップレベルのキーの順序を統一することで,ファイルの一貫性を保ちます.

### ## 事前準備 📝

* `Step 2`の`sbom-tool`によるSBOM生成が完了している必要があります.
* **`generated_sboms/<リポジトリ名>/source/_manifest`** ディレクトリ内に,`manifest.spdx.json`が存在している必要があります.

### ## 実行内容 ⚙️

以下のコードは,`generated_sboms`内の各リポジトリを巡回し,以下の処理を自動的に実行します.

1.  `source/_manifest/spdx_2.2/manifest.spdx.json` を **`combined_sbom.json`** として各リポジトリのルートにコピーします.
2.  コピー直後に`combined_sbom.json`を再度開き,定義された`key_order`リストの順序に従ってキーを並び替えます.
3.  並び替えた内容でファイルを上書き保存します.

In [ ]:
# --- 設定項目 ---

# 1. 処理対象の親ディレクトリ
target_directory = 'generated_sboms'

# 2. 生成するファイル名
target_filename = 'combined_sbom.json'

# 3. 並び替えたいキーの順番
key_order = [
    "SPDXID",
    "spdxVersion",
    "creationInfo",
    "name",
    "dataLicense",
    "documentNamespace",
    "documentDescribes",
    "externalDocumentRefs",
    "packages",
    "files",
    "relationships",
]


# --- 処理の開始 ---
print(f"--- Starting: Copying and reordering SBOMs ---")
print("-" * 50)

try:
    # 'generated_sboms' 内のリポジトリ名を取得
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    
    if not repo_dirs:
         print(f"No repository directories found in '{target_directory}'.")

    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        # --- パスの定義 ---
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', '_manifest', 'spdx_2.2', 'manifest.spdx.json')
        destination_sbom_path = os.path.join(target_directory, repo_name, target_filename)

        # --- スキップ判定 ---
        if os.path.exists(destination_sbom_path):
            print(f"🟢 Skipping: '{target_filename}' already exists.")
            print("-" * 50)
            continue
        
        # --- コピー元の存在確認 ---
        if not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: sbom-tool SBOM not found for {repo_name}. Skipping.")
            print("-" * 50)
            continue

        try:
            # --- ステップ1: コピー ---
            print(f"   Copying sbom-tool's output to '{target_filename}'...")
            shutil.copy(source_sbom_path, destination_sbom_path)
            print("   ✅ Copy successful.")

            # --- ステップ2: 読み込みと並び替え ---
            with open(destination_sbom_path, 'r', encoding='utf-8') as f:
                original_data = json.load(f)

            ordered_data = {}
            # 指定された順でキーを追加
            for key in key_order:
                if key in original_data:
                    ordered_data[key] = original_data[key]
            # 残りのキーを末尾に追加
            for key, value in original_data.items():
                if key not in ordered_data:
                    ordered_data[key] = value

            # --- ステップ3: 整形して上書き保存 ---
            print(f"   Reordering keys...")
            with open(destination_sbom_path, 'w', encoding='utf-8') as f:
                json.dump(ordered_data, f, indent=2, ensure_ascii=False)
            print(f"   ✅ Keys reordered and saved.")

        except json.JSONDecodeError:
            print(f"❌ Error: Could not parse source JSON file. It may be corrupted.")
        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
        
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The source directory '{target_directory}' was not found.")

print("All processes finished.")

# Step 7: SBOMの統合 (GitHub Dependency Graph)

このステップでは,`Step 6`で作成したベースSBOM (`combined_sbom.json`) に対して,`Step 4`でGitHub APIから取得した`dependency-graph-sbom.json`の情報を統合します.この処理により,特に**ライセンス**や**著作権**に関する情報が大幅に補強されます.

### ## 実行内容 ⚙️

以下のコードは,2つのSBOMファイルを比較し,`combined_sbom.json`をより完全なものにするために,以下の4つの主要な処理を実行します.

#### 1. 既存パッケージ情報の補完
`purl`（Package URL）を基準に2つのSBOM間でパッケージを照合し,`combined_sbom.json`のパッケージ情報に不足があれば,`dependency-graph-sbom.json`から以下の情報を補完します.

* **`licenseConcluded`**: ライセンス情報が`NOASSERTION`の場合に更新します.
* **`copyrightText`**: 著作権情報が`NOASSERTION`の場合に更新します.

#### 2. 不足パッケージの追加
`sbom-tool`では検出されなかったが,`dependency-graph`では検出されたパッケージ（主にGitHub Actionsなど）を追加します.重複を防ぐため,**`name`（パッケージ名）**が`combined_sbom.json`に存在しないもののみが追加対象となります.

#### 3. `creators`情報の追記
`creationInfo`セクションに,`dependency-graph`を生成したツールの情報（`Tool: ...`）を追記します.

#### 4. ドキュメントコメントの追記
ファイル全体に関する`comment`フィールドに,`dependency-graph`からの注釈を追記します.

In [ ]:
def get_purl_from_package(pkg):
    """パッケージ情報からpurl（Package URL）を抽出する."""
    if 'externalRefs' in pkg:
        for ref in pkg['externalRefs']:
            if ref.get('referenceType') == 'purl':
                return ref.get('referenceLocator')
    return None

# --- 設定項目 ---
target_directory = 'generated_sboms'
base_sbom_filename = 'combined_sbom.json'
source_sbom_filename = 'dependency-graph-sbom.json'


# --- 処理の開始 ---
print(f"--- Starting: Supplementing '{base_sbom_filename}' with richer data and comments ---")
print("-" * 50)

try:
    # 'generated_sboms' ディレクトリ内のリポジトリ名一覧を取得する
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]

    if not repo_dirs:
        print(f"No repository directories found in '{target_directory}'.")

    # 各リポジトリに対して処理を実行する
    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        # --- パスの定義 ---
        base_sbom_path = os.path.join(target_directory, repo_name, base_sbom_filename)
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', source_sbom_filename)

        # --- ファイルの存在確認 ---
        if not os.path.exists(base_sbom_path) or not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: One or both SBOM files are missing. Skipping.")
            print("-" * 50)
            continue

        try:
            # --- ステップ1: 両方のSBOMファイルを読み込む ---
            with open(base_sbom_path, 'r', encoding='utf-8') as f:
                base_data = json.load(f)
            with open(source_sbom_path, 'r', encoding='utf-8') as f:
                source_data = json.load(f)

            changes_made = 0

            # --- ステップ2: 既存パッケージの情報を補完 ---
            # 補完元パッケージ情報をpurlをキーにマップ化し,検索を高速化する
            source_package_map = {}
            if 'packages' in source_data:
                for pkg in source_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl:
                        source_package_map[purl] = {
                            'licenseConcluded': pkg.get('licenseConcluded', 'NOASSERTION'),
                            'copyrightText': pkg.get('copyrightText', 'NOASSERTION')
                        }

            # ベースSBOMのパッケージ情報を順に確認し,不足していれば補完する
            if 'packages' in base_data:
                for pkg in base_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl and purl in source_package_map:
                        source_pkg_info = source_package_map[purl]

                        if pkg.get('licenseConcluded') == 'NOASSERTION' and source_pkg_info['licenseConcluded'] != 'NOASSERTION':
                            pkg['licenseConcluded'] = source_pkg_info['licenseConcluded']
                            changes_made += 1
                            print(f"   Updated license for: {pkg.get('name')}")

                        if pkg.get('copyrightText') == 'NOASSERTION' and source_pkg_info['copyrightText'] != 'NOASSERTION':
                            pkg['copyrightText'] = source_pkg_info['copyrightText']
                            changes_made += 1
                            print(f"   Updated copyright for: {pkg.get('name')}")
            
            # --- ステップ3: 不足しているパッケージを「パッケージ名」で判定して追加 ---
            if 'packages' in source_data:
                # ベースSBOMのパッケージ名をセットに格納し,検索を高速化する
                base_pkg_names = {pkg.get('name') for pkg in base_data.get('packages', []) if pkg.get('name')}
                new_pkgs_added = 0
                
                for source_pkg in source_data.get('packages', []):
                    pkg_name = source_pkg.get('name')
                    # パッケージ名がベースSBOMに存在しない場合のみ追加する
                    if pkg_name and pkg_name not in base_pkg_names:
                        base_data.setdefault('packages', []).append(source_pkg)
                        base_pkg_names.add(pkg_name) # 追加した名前をセットに加え,重複追加を防ぐ
                        new_pkgs_added += 1
                        print(f"   Added new package from dependency-graph: {pkg_name}")
                
                if new_pkgs_added > 0:
                    changes_made += new_pkgs_added
                    print(f"   A total of {new_pkgs_added} new packages were added.")

            # --- ステップ4: creationInfo.creators の情報を追記 ---
            if 'creationInfo' in source_data and 'creators' in source_data['creationInfo'] and \
               'creationInfo' in base_data and 'creators' in base_data['creationInfo']:
                base_creators = base_data['creationInfo']['creators']
                for creator in source_data['creationInfo']['creators']:
                    if creator not in base_creators:
                        base_creators.append(creator)
                        changes_made += 1
                        print(f"   Added creator: {creator}")

            # --- ステップ5: コメントを追記 ---
            source_doc_comment = source_data.get('comment')
            if source_doc_comment:
                comment_header = "Note from dependency-graph"
                full_comment_to_add = f"{comment_header}: {source_doc_comment}"
                if 'comment' not in base_data or not base_data.get('comment'):
                    base_data['comment'] = full_comment_to_add
                    changes_made += 1
                    print("   Added document-level comment.")
                elif full_comment_to_add not in base_data['comment']:
                    base_data['comment'] += f"\\n\\n{full_comment_to_add}"
                    changes_made += 1
                    print("   Appended document-level comment.")

            # --- ステップ6: 変更があった場合のみファイルを上書き保存 ---
            if changes_made > 0:
                print(f"   {changes_made} fields were updated/added. Reordering and saving file...")

                # キーの順序を定義し,それに従ってファイルを再構築する
                key_order = [
                    "SPDXID", "spdxVersion", "creationInfo", "name", "dataLicense",
                    "documentNamespace", "comment", "documentDescribes", "externalDocumentRefs",
                    "packages", "files", "relationships"
                ]
                ordered_data = {key: base_data[key] for key in key_order if key in base_data}
                ordered_data.update({key: value for key, value in base_data.items() if key not in ordered_data})

                with open(base_sbom_path, 'w', encoding='utf-8') as f:
                    json.dump(ordered_data, f, indent=2, ensure_ascii=False)
                print(f"✅ Successfully supplemented '{base_sbom_filename}'.")
            else:
                print("   No fields needed updating or adding.")

        except json.JSONDecodeError as e:
            print(f"❌ Error: Could not parse a JSON file. Details: {e}")
        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The top-level directory '{target_directory}' was not found.")

print("All processes finished.")

# Step 8: SBOMの統合 (Syft)

`sbom-tool`とGitHub APIの情報を統合した`combined_sbom.json`に対し,さらに`syft`が検出した情報をマージしてSBOMを完成させます.`syft`は,パッケージの提供者（`supplier`）や作成元（`originator`）,検出場所（`sourceInfo`）,そして脆弱性スキャンに不可欠な**CPE**（Common Platform Enumeration）などの詳細情報を検出する能力に優れています.

### ## 実行内容 ⚙️

以下のコードは,2つのSBOMファイルを比較し,`combined_sbom.json`をさらにリッチなものにするために,以下の4つの主要な処理を実行します.

#### 1. 既存パッケージ情報の補完
`purl`を基準にパッケージを照合し,`combined_sbom.json`のパッケージ情報に不足があれば,`syft-sbom.json`から以下の情報を補完します.

* **`supplier`** と **`originator`**: `NOASSERTION`の場合に更新します.
* **`sourceInfo`**: 項目が存在しない場合に追加します.
* **`externalRefs`**: `purl`や`cpe`などの外部リンクが重複しないように追記します.

#### 2. 不足パッケージの追加
`sbom-tool`やGitHub APIでは検出されなかったが,`syft`では検出されたパッケージを追加します.重複を防ぐため,**`name`（パッケージ名）**が`combined_sbom.json`に存在しないもののみが追加対象となります.

#### 3. `creators`情報の追記
`creationInfo`セクションに,`syft`を生成したツールの情報（`Tool: syft-...`）を追記します.

#### 4. 整形して保存
最後に,すべての情報が統合された`combined_sbom.json`のキーの順序を統一的なフォーマットに整え,ファイルを上書き保存します.

In [ ]:
def get_purl_from_package(pkg):
    """パッケージ情報からpurl（Package URL）を抽出する."""
    if 'externalRefs' in pkg:
        for ref in pkg['externalRefs']:
            if ref.get('referenceType') == 'purl':
                return ref.get('referenceLocator')
    return None

# --- 設定項目 ---
target_directory = 'generated_sboms'
base_sbom_filename = 'combined_sbom.json'
source_sbom_filename = 'syft-sbom.json'

# --- 処理の開始 ---
print(f"--- Starting: Supplementing '{base_sbom_filename}' with data from syft (using name for matching) ---")
print("-" * 50)

try:
    # 'generated_sboms' 内のリポジトリ名を取得する
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    
    if not repo_dirs:
        print(f"No repository directories found in '{target_directory}'.")

    # 各リポジトリに対して処理を実行する
    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        # --- パスの定義 ---
        base_sbom_path = os.path.join(target_directory, repo_name, base_sbom_filename)
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', source_sbom_filename)

        # --- ファイルの存在確認 ---
        if not os.path.exists(base_sbom_path) or not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: One or both SBOM files are missing. Skipping.")
            print("-" * 50)
            continue

        try:
            # --- ステップ1: 両方のSBOMファイルを読み込む ---
            with open(base_sbom_path, 'r', encoding='utf-8') as f:
                base_data = json.load(f)
            with open(source_sbom_path, 'r', encoding='utf-8') as f:
                source_data = json.load(f)

            # 変更を検出するため,処理前のデータを文字列として保存する
            initial_data_str = json.dumps(base_data, sort_keys=True)

            # --- ステップ2: syftのパッケージ情報をpurlをキーにした辞書に整理 ---
            source_package_map = {}
            if 'packages' in source_data:
                for pkg in source_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl and purl not in source_package_map:
                        source_package_map[purl] = pkg

            # --- ステップ3: 既存パッケージの情報を補完 ---
            base_purls = set()
            if 'packages' in base_data:
                for pkg in base_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl:
                        base_purls.add(purl)
                        if purl in source_package_map:
                            source_pkg = source_package_map[purl]
                            
                            # サプライヤー情報を補完
                            if (not pkg.get('supplier') or pkg.get('supplier') == 'NOASSERTION') and \
                               source_pkg.get('supplier') and source_pkg.get('supplier') != 'NOASSERTION':
                                pkg['supplier'] = source_pkg['supplier']
                            
                            # 作成元情報を補完
                            if (not pkg.get('originator') or pkg.get('originator') == 'NOASSERTION') and \
                               source_pkg.get('originator') and source_pkg.get('originator') != 'NOASSERTION':
                                pkg['originator'] = source_pkg['originator']

                            # 検出元情報を補完
                            if 'sourceInfo' not in pkg and source_pkg.get('sourceInfo'):
                                pkg['sourceInfo'] = source_pkg['sourceInfo']
                            
                            # 外部リンク(CPEなど)を追記
                            if 'externalRefs' not in pkg: pkg['externalRefs'] = []
                            existing_locators = {ref.get('referenceLocator') for ref in pkg['externalRefs']}
                            for new_ref in source_pkg.get('externalRefs', []):
                                if new_ref.get('referenceLocator') not in existing_locators:
                                    pkg['externalRefs'].append(new_ref)

            # --- ステップ4: 不足しているパッケージを「パッケージ名」で判定して追加 ---
            if 'packages' in source_data:
                # ベースSBOMのパッケージ名をセットに格納し,検索を高速化する
                base_pkg_names = {pkg.get('name') for pkg in base_data.get('packages', []) if pkg.get('name')}
                
                for source_pkg in source_data.get('packages', []):
                    pkg_name = source_pkg.get('name')
                    # パッケージ名がベースSBOMに存在しない場合のみ追加する
                    if pkg_name and pkg_name not in base_pkg_names:
                        base_data.setdefault('packages', []).append(source_pkg)
                        base_pkg_names.add(pkg_name) # 追加した名前をセットに加え,重複追加を防ぐ

            # --- ステップ5: syftのツール情報をcreatorsに追加 ---
            if 'creationInfo' in source_data and 'creators' in source_data['creationInfo']:
                base_creators = base_data.setdefault('creationInfo', {}).setdefault('creators', [])
                for creator in source_data['creationInfo']['creators']:
                    if creator not in base_creators:
                        base_creators.append(creator)
            
            # --- ステップ6: 変更があった場合のみファイルを保存 ---
            if initial_data_str != json.dumps(base_data, sort_keys=True):
                print(f"   Changes detected. Reordering keys and saving file...")

                # キーの順序を定義し,それに従ってファイルを再構築する
                key_order = [
                    "SPDXID", "spdxVersion", "creationInfo", "name", "dataLicense",
                    "documentNamespace", "comment", "documentDescribes", "externalDocumentRefs",
                    "packages", "files", "relationships"
                ]
                ordered_data = {key: base_data[key] for key in key_order if key in base_data}
                ordered_data.update({key: value for key, value in base_data.items() if key not in ordered_data})

                with open(base_sbom_path, 'w', encoding='utf-8') as f:
                    json.dump(ordered_data, f, indent=2, ensure_ascii=False)
                print(f"✅ Successfully supplemented and reordered '{base_sbom_filename}'.")
            else:
                print("   No new information to supplement from syft.")

        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Error processing files for {repo_name}. Details: {e}")
        
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The top-level directory '{target_directory}' was not found.")

print("All processes finished.")

# Step 9: SBOMの統合 (Trivy)

このステップでは,これまでの処理で情報を充実させてきた`combined_sbom.json`に対し,最後に`Trivy`が生成した`trivy-sbom.json`の情報を統合します.`Trivy`は,パッケージの目的 (`primaryPackagePurpose`) や,ツール固有の注釈 (`annotations`) といった,他のツールでは得られないユニークな情報を提供します.

### ## 実行内容 ⚙️

以下のコードは,`combined_sbom.json`を最終的に完成させるため,以下の4つの主要な処理を実行します.

#### 1. 既存パッケージ情報の補完
`purl`を基準にパッケージを照合し,`combined_sbom.json`のパッケージ情報に不足があれば,`trivy-sbom.json`から以下の情報を補完します.

* **`primaryPackagePurpose`**: パッケージの主な目的（例: `LIBRARY`, `APPLICATION`）が`NOASSERTION`または未設定の場合に更新します.
* **`annotations`**: `Trivy`による注釈情報（例: `PkgType: pip`）が未設定の場合に追加します.

#### 2. 不足パッケージの追加
これまでのツールでは検出されなかったが,`Trivy`が独自に検出したパッケージを追加します.重複を防ぐため,**`name`（パッケージ名）**が`combined_sbom.json`に存在しないもののみが追加対象となります.

#### 3. `creators`情報の追記
`creationInfo`セクションに,`Trivy`を生成したツールの情報（`Tool: trivy-...`）を追記します.

#### 4. 整形して保存
最後に,すべての情報が統合された`combined_sbom.json`のキーの順序を統一的なフォーマットに整え,ファイルを上書き保存します.

In [ ]:
def get_purl_from_package(pkg):
    """パッケージ情報からpurl（Package URL）を抽出する."""
    if 'externalRefs' in pkg:
        for ref in pkg['externalRefs']:
            if ref.get('referenceType') == 'purl':
                return ref.get('referenceLocator')
    return None

# --- 設定項目 ---
target_directory = 'generated_sboms'
base_sbom_filename = 'combined_sbom.json'
# Trivyで生成したファイル名を指定
source_sbom_filename = 'trivy-sbom.json'

# --- 処理の開始 ---
print(f"--- Starting: Supplementing '{base_sbom_filename}' with data from Trivy ---")
print("-" * 50)

try:
    # 'generated_sboms' 内のリポジトリ名を取得する
    repo_dirs = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
    
    if not repo_dirs:
        print(f"No repository directories found in '{target_directory}'.")

    # 各リポジトリに対して処理を実行する
    for repo_name in repo_dirs:
        print(f"▶️  Processing: {repo_name}")

        # --- パスの定義 ---
        base_sbom_path = os.path.join(target_directory, repo_name, base_sbom_filename)
        source_sbom_path = os.path.join(target_directory, repo_name, 'source', source_sbom_filename)

        # --- ファイルの存在確認 ---
        if not os.path.exists(base_sbom_path) or not os.path.exists(source_sbom_path):
            print(f"⚠️ Warning: One or both SBOM files are missing. Skipping.")
            print("-" * 50)
            continue

        try:
            # --- ステップ1: 両方のSBOMファイルを読み込む ---
            with open(base_sbom_path, 'r', encoding='utf-8') as f:
                base_data = json.load(f)
            with open(source_sbom_path, 'r', encoding='utf-8') as f:
                source_data = json.load(f)

            initial_data_str = json.dumps(base_data, sort_keys=True)

            # --- ステップ2: Trivyのパッケージ情報をpurlをキーにした辞書に整理 ---
            source_package_map = {}
            if 'packages' in source_data:
                for pkg in source_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl and purl not in source_package_map:
                        source_package_map[purl] = pkg

            # --- ステップ3: 既存パッケージの情報を補完 ---
            if 'packages' in base_data:
                for pkg in base_data['packages']:
                    purl = get_purl_from_package(pkg)
                    if purl and purl in source_package_map:
                        source_pkg = source_package_map[purl]
                        
                        # primaryPackagePurposeを補完
                        if (not pkg.get('primaryPackagePurpose') or pkg.get('primaryPackagePurpose') == 'NOASSERTION') and \
                           source_pkg.get('primaryPackagePurpose') and source_pkg.get('primaryPackagePurpose') != 'NOASSERTION':
                            pkg['primaryPackagePurpose'] = source_pkg['primaryPackagePurpose']
                            print(f"   Updated primaryPackagePurpose for: {pkg.get('name')}")

                        # annotationsを追記
                        if 'annotations' not in pkg and source_pkg.get('annotations'):
                            pkg['annotations'] = source_pkg['annotations']
                            print(f"   Added annotations for: {pkg.get('name')}")

            # --- ステップ4: 不足しているパッケージを「パッケージ名」で判定して追加 ---
            if 'packages' in source_data:
                # ベースSBOMのパッケージ名をセットに格納し,検索を高速化する
                base_pkg_names = {pkg.get('name') for pkg in base_data.get('packages', []) if pkg.get('name')}
                
                for source_pkg in source_data.get('packages', []):
                    pkg_name = source_pkg.get('name')
                    # パッケージ名がベースSBOMに存在しない場合のみ追加する
                    if pkg_name and pkg_name not in base_pkg_names:
                        base_data.setdefault('packages', []).append(source_pkg)
                        base_pkg_names.add(pkg_name)
                        print(f"   Added new package from Trivy: {pkg_name}")

            # --- ステップ5: Trivyのツール情報をcreatorsに追加 ---
            if 'creationInfo' in source_data and 'creators' in source_data['creationInfo']:
                base_creators = base_data.setdefault('creationInfo', {}).setdefault('creators', [])
                for creator in source_data['creationInfo']['creators']:
                    if creator not in base_creators:
                        base_creators.append(creator)
            
            # --- ステップ6: 変更があった場合のみファイルを保存 ---
            if initial_data_str != json.dumps(base_data, sort_keys=True):
                print(f"   Changes detected. Reordering keys and saving file...")

                # キーの順序を定義し,それに従ってファイルを再構築する
                key_order = [
                    "SPDXID", "spdxVersion", "creationInfo", "name", "dataLicense",
                    "documentNamespace", "comment", "documentDescribes", "externalDocumentRefs",
                    "packages", "files", "relationships"
                ]
                ordered_data = {key: base_data[key] for key in key_order if key in base_data}
                ordered_data.update({key: value for key, value in base_data.items() if key not in ordered_data})

                with open(base_sbom_path, 'w', encoding='utf-8') as f:
                    json.dump(ordered_data, f, indent=2, ensure_ascii=False)
                print(f"✅ Successfully supplemented and reordered '{base_sbom_filename}'.")
            else:
                print("   No new information to supplement from Trivy.")

        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Error processing files for {repo_name}. Details: {e}")
        
        finally:
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The top-level directory '{target_directory}' was not found.")

print("All processes finished.")